In [1]:
from scripts.processing import predict, load_audio
import onnxruntime as ort
import matplotlib.pyplot as plt
import numpy as np
import librosa as lb
import matplotlib.pyplot as plt


In [2]:
model_type="CRNN" #select model YOLO / CRNN
providers=['CPUExecutionProvider']

if model_type=="YOLO":
  ort_sess = ort.InferenceSession('models/YOLOv5n.onnx', providers=providers)
elif model_type=="CRNN":
  ort_sess = ort.InferenceSession('models/CRNN.onnx',providers=providers)

In [3]:
th=0.2 #Threshold of the confidence of the prediction
Audio_dir = "samples/grabacion_mic0_0.wav" #Audio to predict
dir_audio=Audio_dir
sr=16000
audio=load_audio(dir_audio,sr)
classes=["Horn","Siren","Pets","Physiological","Speech","RingTone","Vibrating","Notifications","Cry"] #List of predictions
p,_=predict(audio, ort_sess,model_type,th) #Do the prediction
print(p)

[array([0.        , 3.14102564, 3.        , 0.51371366]), array([5.51282051, 5.96153846, 3.        , 0.51371366]), array([ 7.82051282, 10.        ,  3.        ,  0.51371366]), array([4.74358974, 5.51282051, 4.        , 0.24481095]), array([2.69230769, 3.07692308, 6.        , 0.39921179]), array([5.76923077, 7.94871795, 6.        , 0.39921179]), array([9.23076923, 9.35897436, 6.        , 0.39921179])]
